<a href="https://colab.research.google.com/github/tozanni/hdw_interpretability/blob/main/LogisticRegression_p_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data prep thanks to:
https://www.kaggle.com/code/renadope/cervical-cancer-classification-99-4-recall


In [15]:
%pip install statsmodels
#%pip install mlxtend

In [16]:
import pandas as pd
import numpy as np
risk_factor_df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00383/risk_factors_cervical_cancer.csv")
#risk_factor_df.head()



In [17]:
from sklearn.impute import SimpleImputer

#these columns are not of type object, but are of type numeric
cols_to_convert = ['Number of sexual partners', 'First sexual intercourse', 'Num of pregnancies', 'Smokes',
                   'Smokes (years)', 'Smokes (packs/year)', 'Hormonal Contraceptives',
                   'Hormonal Contraceptives (years)', 'IUD', 'IUD (years)', 'STDs', 'STDs (number)',
                   'STDs:condylomatosis', 'STDs:cervical condylomatosis', 'STDs:vaginal condylomatosis',
                   'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis', 'STDs:pelvic inflammatory disease',
                   'STDs:genital herpes', 'STDs:molluscum contagiosum', 'STDs:AIDS', 'STDs:HIV', 'STDs:Hepatitis B',
                   'STDs:HPV', 'STDs: Time since first diagnosis',
                   'STDs: Time since last diagnosis']
# for i in range(0,len(cols_to_convert)):
#     print("{}={}".format(i,cols_to_convert[i]))
risk_factor_df[cols_to_convert] = risk_factor_df[cols_to_convert].apply(pd.to_numeric, errors="coerce")
risk_factor_df[cols_to_convert].fillna(np.nan, inplace=True)
imp = SimpleImputer(strategy="median")
X = imp.fit_transform(risk_factor_df)
risk_factor_df = pd.DataFrame(X, columns=list(risk_factor_df.columns))

<ipython-input-17-b33c7bd60fed>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  risk_factor_df[cols_to_convert].fillna(np.nan, inplace=True)


In [18]:
risk_factor_df["Age"] = risk_factor_df["Age"].astype(int)

In [19]:
std_cols = {'STDs:condylomatosis','STDs:cervical condylomatosis','STDs:vaginal condylomatosis',
            'STDs:vulvo-perineal condylomatosis','STDs:syphilis','STDs:pelvic inflammatory disease',
            'STDs:genital herpes','STDs:molluscum contagiosum','STDs:AIDS',
            'STDs:HIV','STDs:Hepatitis B','STDs:HPV'}

#risk_factor_df["total_std"] = risk_factor_df[list(std_cols)].sum(axis=1)
#std_agg = risk_factor_df.groupby("age_cat", as_index=False)[list(std_cols)].sum()

In [20]:
test_cols = ["Hinselmann", "Schiller", "Citology", "Biopsy"]
risk_factor_df["total_tests"] = risk_factor_df[test_cols].sum(axis = 1)

In [21]:
to_int_and_beyond = {"total_tests",
                     #"total_std",
                     "Smokes","Biopsy","Dx:Cancer","Num of pregnancies","Number of sexual partners","First sexual intercourse",
                     "Hormonal Contraceptives","IUD","STDs","STDs (number)","STDs: Number of diagnosis","Dx:CIN","Dx:HPV","Dx","Hinselmann",
                     "Schiller","Biopsy","Citology"}

to_int_and_beyond = to_int_and_beyond.union(std_cols)

for col in to_int_and_beyond:
    risk_factor_df[col] = risk_factor_df[col].astype(int)

In [22]:
y = risk_factor_df["Dx:Cancer"].copy()

X = risk_factor_df[["Number of sexual partners","Hormonal Contraceptives","Smokes","Num of pregnancies","STDs (number)","IUD"]]
#X = risk_factor_df.drop(risk_factor_df.iloc[:, 13:36],axis = 1)

In [23]:
from imblearn.over_sampling import ADASYN, SMOTE
#adasyn = ADASYN(random_state=42)
#X_adasyn, y_adasyn = adasyn.fit_resample(X,y)

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X,y)


In [24]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_adasyn, y_adasyn, test_size=0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

In [25]:
X.corr()

,Number of sexual partners,Hormonal Contraceptives,Smokes,Num of pregnancies,STDs (number),IUD
Number of sexual partners,1.000000,0.004027,0.236858,0.077439,0.041459,0.032460
Hormonal Contraceptives,0.004027,1.000000,0.004036,0.118938,-0.053642,0.000188
Smokes,0.236858,0.004036,1.000000,0.081517,0.100117,-0.055115
Num of pregnancies,0.077439,0.118938,0.081517,1.000000,0.010603,0.204501
STDs (number),0.041459,-0.053642,0.100117,0.010603,1.000000,0.060591
IUD,0.032460,0.000188,-0.055115,0.204501,0.060591,1.000000


In [26]:
import statsmodels.api as sm
X_train = sm.add_constant(X_train)
log_reg_model = sm.Logit(y_train, X_train)
log_reg_model = log_reg_model.fit()
print(log_reg_model.summary())

Optimization terminated successfully.
         Current function value: 0.625872
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:              Dx:Cancer   No. Observations:                 1344
Model:                          Logit   Df Residuals:                     1337
Method:                           MLE   Df Model:                            6
Date:                Thu, 24 Aug 2023   Pseudo R-squ.:                 0.09701
Time:                        06:42:08   Log-Likelihood:                -841.17
converged:                       True   LL-Null:                       -931.54
Covariance Type:            nonrobust   LLR p-value:                 2.375e-36
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.1426      0.182      0.782      0.434      -0.

In [27]:
from sklearn.metrics import accuracy_score, classification_report

X_test = sm.add_constant(X_test)
y_pred = log_reg_model.predict(X_test) > 0.5

print("Accuracy is ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy is  0.5833333333333334
              precision    recall  f1-score   support

           0       0.63      0.47      0.54       174
           1       0.55      0.70      0.62       162

    accuracy                           0.58       336
   macro avg       0.59      0.59      0.58       336
weighted avg       0.59      0.58      0.58       336

